#### Definir a distancia entre filmes baseado em seus metadados utilizando a tecnica KNN

In [80]:
import pandas as pd
import numpy as np
from scipy import spatial
import operator

In [17]:
r_cols = ['user_id', 'movie_id', 'rating']
# o arquivo csv não tem as informações separadas por virgula, mas sim por tabulações
ratings = pd.read_csv(r"Aquivos_de_Código\u.data", sep='\t', names=r_cols, usecols=range(3))

In [16]:
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [33]:
# mostra a quantidade de avaliações e a nota por filme
movieProperties = ratings.groupby('movie_id').agg({'rating': ['size', 'mean']})

In [34]:
movieProperties

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326
...         ...       ...
1678          1  1.000000
1679          1  3.000000
1680          1  2.000000
1681          1  3.000000
1682          1  3.000000

[1682 rows x 2 columns]

In [28]:
movieProperties['rating']['size']

movie_id
1       452
2       131
3        90
4       209
5        86
       ... 
1678      1
1679      1
1680      1
1681      1
1682      1
Name: size, Length: 1682, dtype: int64

In [36]:

# para obter informações mais consistentes, pode-se normalizar as notas 
# com o numero de avaliações para variar de 0 a 1, sendo 0 ninguém avaliou e 1 com o maior número de avaliações
 
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
 

In [37]:
movieNumRatings

,size
movie_id,
1,452
2,131
3,90
4,209
5,86
...,...
1678,1
1679,1
1680,1


In [38]:
#para normalizar, aplica-se na coluna size do df uma função para que o valor máximo seja 1 e o valor mínimo seja 0.
# por exemplo 

movieNormalizeNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [41]:
movieNormalizeNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


In [65]:
#carregar as informações de cada filme

movieDict = {}

with open(r"Aquivos_de_Código\u.item") as file:
    temp = ''
    for line in file:
        fields = line.rstrip('\n').split('|') # lista de informações -> rstrip(remove a direta, no caso '\n') e split(separa os elementos quando encontra um '|')
        # print(fields)
        movieID = int(fields[0])
        name = fields[1]
        genres = list(map(int,fields[5:25]))
        movieDict[movieID] = (name, genres, movieNormalizeNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))
    

In [63]:
movieDict[1]

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)

In [74]:
# para calcular as distancias com o metodo 'distance.cosine' com 0 indicando que os vetores são idênticos e 1 indicando que são completamente diferentes.
def distance(a,b):
    genreA = a[1]
    genreB = b[1]
    genreDistance = spatial.distance.cosine(genreA, genreB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance

In [75]:
distance(movieDict[2], movieDict[4])

0.8004574042309892

In [73]:
print(movieDict[1], '\n', movieDict[4])
# Sao filmes bem diferentes, porem o score deu relativamente alto

('Toy Story (1995)', [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.7735849056603774, 3.8783185840707963) 
 ('Get Shorty (1995)', [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.3567753001715266, 3.550239234449761)


In [108]:
# quais são os k filmes mais proximos de um filme

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = distance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))# itemgetter(1) é uma função que extrai o segundo elemento de cada tupla (nesse caso as distancias) para usar como chave de ordenação.
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors
            

In [113]:
K = 10
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print(movieDict[neighbor][0], ' ', str(movieDict[neighbor][3])) # filmes infantis também
    
avgRating /= float(K)

Liar Liar (1997)   3.156701030927835
Aladdin (1992)   3.8127853881278537
Willy Wonka and the Chocolate Factory (1971)   3.6319018404907975
Monty Python and the Holy Grail (1974)   4.0664556962025316
Full Monty, The (1997)   3.926984126984127
George of the Jungle (1997)   2.685185185185185
Beavis and Butt-head Do America (1996)   2.7884615384615383
Birdcage, The (1996)   3.4436860068259385
Home Alone (1990)   3.0875912408759123
Aladdin and the King of Thieves (1996)   2.8461538461538463
